In [1]:
!pip install opendatasets
import opendatasets as od

In [2]:
od.download("https://www.kaggle.com/datasets/anshtanwar/microscopic-fungi-images")

100%|██████████| 149M/149M [00:01<00:00, 81.9MB/s]


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the folder paths and class labels
data_folder = "/content/microscopic-fungi-images"  # Replace this with the actual path to your dataset folder
class_labels = {
    'H0': "Candida albicans",
    'H1': "Aspergillus niger",
    "H2": "Trichophyton rubrum",
    "H5": "Trichophyton mentagrophytes",
    "H6": "Epidermophyton floccosum",
}

# Data augmentation and normalization for training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Data normalization for validation and test sets
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Define the batch size and image dimensions
batch_size = 32
image_height, image_width = 128, 128

# Create data generators for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=list(class_labels.keys()),
    shuffle=True,
)

val_generator = val_test_datagen.flow_from_directory(
    data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=list(class_labels.keys()),
    shuffle=False,
)

test_generator = val_test_datagen.flow_from_directory(
    data_folder,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=list(class_labels.keys()),
    shuffle=False,
)

# Load the pre-trained VGG16 model without the top (classification) layers
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(image_height, image_width, 3))

# Add a custom classifier on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(len(class_labels), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model to keep their pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
epochs = 10
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
)

# Evaluate the model on the test set
test_steps = len(test_generator)
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Found 8295 images belonging to 5 classes.
Found 8295 images belonging to 5 classes.
Found 8295 images belonging to 5 classes.
58889256/58889256 [==============================] - 0s 0us/step


Epoch 1/10
260/260 [==============================] - 75s 245ms/step - loss: 0.9059 - accuracy: 0.6171 - val_loss: 0.8171 - val_accuracy: 0.6517
Epoch 2/10
260/260 [==============================] - 63s 241ms/step - loss: 0.7980 - accuracy: 0.6621 - val_loss: 0.7845 - val_accuracy: 0.6634
Epoch 3/10
260/260 [==============================] - 63s 241ms/step - loss: 0.7665 - accuracy: 0.6708 - val_loss: 0.7588 - val_accuracy: 0.6732
Epoch 4/10
260/260 [==============================] - 62s 239ms/step - loss: 0.7429 - accuracy: 0.6839 - val_loss: 0.7575 - val_accuracy: 0.6887
Epoch 5/10
260/260 [==============================] - 63s 241ms/step - loss: 0.7248 - accuracy: 0.6856 - val_loss: 0.6933 - val_accuracy: 0.7049
Epoch 6/10
260/260 [==============================] - 64s 244ms/step - loss: 0.7087 - accuracy: 0.6986 - val_loss: 0.6685 - val_accuracy: 0.7107
Epoch 7/10
260/260 [==============================] - 63s 241ms/step - loss: 0.6942 - accuracy: 0.7008 - val_loss: 0.6774 - val_ac